In [46]:
import rasterio, matplotlib.pyplot as plt, pandas as pd, numpy as np

In [47]:
image_file = ('/home/dev103/ownCloud/PLANET_CHALLENGE/Planet/litchfieldData/2021-08-31_7f175324-5ded-4cf8-801b-bae9d60774cd/files/composite.tif')

In [48]:
# Load bands - note all PlanetScope 4-band images have band order BGRN
with rasterio.open(image_file) as src:
    band_blue = src.read(1)
    band_green = src.read(2)
    band_red = src.read(3)
    band_nir = src.read(4)

In [49]:
from xml.dom import minidom

xmldoc = minidom.parse("/home/dev103/ownCloud/PLANET_CHALLENGE/Planet/litchfieldData/2021-08-31_7f175324-5ded-4cf8-801b-bae9d60774cd/files/20210831_005334_40_225a_3B_AnalyticMS_metadata_clip.xml")
nodes = xmldoc.getElementsByTagName("ps:bandSpecificMetadata")

# XML parser refers to bands by numbers 1-4
coeffs = {}
for node in nodes:
    bn = node.getElementsByTagName("ps:bandNumber")[0].firstChild.data
    if bn in ['1', '2', '3', '4']:
        i = int(bn)
        value = node.getElementsByTagName("ps:reflectanceCoefficient")[0].firstChild.data
        coeffs[i] = float(value)

In [50]:
# Multiply by corresponding coefficients = band_red * coeffs[3]
band_blue = band_blue * coeffs[1]
band_green = band_green * coeffs[2]
band_red = band_red * coeffs[3]
band_nir = band_nir * coeffs[4]

In [51]:
# Allow division by zero
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [52]:
# Calculate NDVI
NDVI = (band_nir.astype(float) - band_red.astype(float)) / (band_nir + band_red)

# Set spatial characteristics of the output object to mirror the input
kwargs = src.meta
kwargs.update(
    dtype=rasterio.float32,
    count = 1)

# Create the file
with rasterio.open('NDVI.tif', 'w', **kwargs) as dst:
        dst.write_band(1, NDVI.astype(rasterio.float32))

plt.imsave("NDVI_cmap.png", NDVI, cmap=plt.cm.summer)

NDVI

array([[0.67973946, 0.6803611 , 0.67287201, ..., 0.63668672, 0.62907257,
        0.60929218],
       [0.68342081, 0.68754733, 0.68543533, ..., 0.62677139, 0.6249371 ,
        0.61718721],
       [0.68474913, 0.68424647, 0.68371433, ..., 0.62546222, 0.62752364,
        0.63038491],
       ...,
       [0.66599836, 0.65524375, 0.65276819, ..., 0.64189163, 0.63751491,
        0.62794908],
       [0.65627676, 0.64611909, 0.63848873, ..., 0.62925186, 0.62446084,
        0.62562674],
       [0.64489259, 0.64541638, 0.647008  , ..., 0.63226768, 0.61810714,
        0.6230811 ]])

In [53]:
# Calculate NDI45
NDI45 = (band_red.astype(float) - band_green.astype(float)) / (band_red.astype(float) + band_green.astype(float))

# Set spatial characteristics of the output object to mirror the input
kwargs = src.meta
kwargs.update(
    dtype=rasterio.float32,
    count = 1)

# Create the file
with rasterio.open('NDI45.tif', 'w', **kwargs) as dst:
        dst.write_band(1, NDI45.astype(rasterio.float32))

plt.imsave("NDI45_cmap.png", NDI45, cmap=plt.cm.summer)

NDI45

array([[0.20883698, 0.19845794, 0.21225093, ..., 0.22269081, 0.23012796,
        0.23823799],
       [0.19610427, 0.18913491, 0.18576807, ..., 0.23126938, 0.22423588,
        0.23068009],
       [0.20967439, 0.19935792, 0.18258493, ..., 0.24103495, 0.22321884,
        0.2168601 ],
       ...,
       [0.20546866, 0.2223167 , 0.23162438, ..., 0.23765003, 0.24010737,
        0.24770305],
       [0.21657429, 0.22702527, 0.24295641, ..., 0.22930198, 0.23163813,
        0.23474702],
       [0.23063473, 0.22723434, 0.23718507, ..., 0.22962218, 0.23371244,
        0.22867286]])

In [54]:
# Calculate RGR
RGR = (band_red.astype(float) / band_green.astype(float))

# Set spatial characteristics of the output object to mirror the input
kwargs = src.meta
kwargs.update(
    dtype=rasterio.float32,
    count = 1)

# Create the file
with rasterio.open('RGR.tif', 'w', **kwargs) as dst:
        dst.write_band(1, RGR.astype(rasterio.float32))

plt.imsave("RGR_cmap.png", RGR, cmap=plt.cm.summer)

RGR

array([[1.52792402, 1.49519032, 1.53887953, ..., 1.57297871, 1.59783431,
        1.62549192],
       [1.48788483, 1.46650155, 1.45630258, ..., 1.60169159, 1.57810324,
        1.59969874],
       [1.53060256, 1.49799511, 1.44673737, ..., 1.63516746, 1.57472775,
        1.55382211],
       ...,
       [1.5172072 , 1.57174096, 1.60289361, ..., 1.62346702, 1.63195078,
        1.65852469],
       [1.55289042, 1.58740672, 1.6418558 , ..., 1.59505013, 1.60294021,
        1.61351481],
       [1.59954546, 1.58810672, 1.62186793, ..., 1.59612873, 1.60998627,
        1.59293354]])

In [55]:
# Calculate GNDVI
ndi45 = (band_nir.astype(float) - band_green.astype(float)) / (band_green.astype(float) + band_nir.astype(float))

# Set spatial characteristics of the output object to mirror the input
kwargs = src.meta
kwargs.update(
    dtype=rasterio.float32,
    count = 1)

# Create the file
with rasterio.open('ndi45.tif', 'w', **kwargs) as dst:
        dst.write_band(1, ndi45.astype(rasterio.float32))

plt.imsave("ndi45_cmap.png", ndi45, cmap=plt.cm.summer)

ndi45

array([[0.77811879, 0.77427417, 0.77450929, ..., 0.7526619 , 0.75054609,
        0.74009984],
       [0.77558044, 0.77579808, 0.77280109, ..., 0.74941133, 0.74480147,
        0.74219843],
       [0.78212974, 0.77754017, 0.77015611, ..., 0.75297932, 0.74621618,
        0.74535148],
       ...,
       [0.76656842, 0.76597904, 0.76823736, ..., 0.76312962, 0.76111661,
        0.75778287],
       [0.76422912, 0.76145069, 0.76307343, ..., 0.75029478, 0.74791401,
        0.75019684],
       [0.76216673, 0.76103652, 0.76655678, ..., 0.75262215, 0.74429869,
        0.74552961]])